In [2]:
from sfcn import SFCN
import numpy as np
import pandas as pd
from volumedatagenerator import VolumeDataGeneratorRegression

In [3]:
col_list = pd.read_csv('fields/subcortical_volumes.csv')['field id'].to_list()

columns=['path']
for col in col_list:
    columns.append(str(col)+'-2.0')

In [4]:
input_dim = [182, 218, 182]
num_output = len(columns)-1
batch_size=4

In [ ]:
train_df = pd.read_csv('csv/split_train.csv', index_col='eid')[columns]
train_gen = VolumeDataGeneratorRegression(
    sample_df=train_df, 
    batch_size=batch_size, 
    num_reg_classes=num_output, 
    dim=input_dim,
    output_preprocessing='quantile')

scaler_instance = train_gen.get_scaler_instance()

In [5]:
valid_df = pd.read_csv('csv/split_valid.csv', index_col='eid')[columns]
valid_gen = VolumeDataGeneratorRegression(
    sample_df=valid_df, 
    batch_size=batch_size, 
    num_reg_classes=num_output, 
    dim=input_dim,
    output_scaler=scaler_instance)

In [11]:
model = SFCN(
    input_dim=[182, 218, 182, 1], 
    output_dim=num_output,
    conv_num_filters=[32, 64, 128, 256, 256, 64], 
    conv_kernel_sizes=[3, 3, 3, 3, 3, 1], 
    conv_strides=[1, 1, 1, 1, 1, 1],
    conv_padding=['same', 'same', 'same', 'same', 'same', 'valid'],
    pooling_size=[2, 2, 2, 2, 2],
    pooling_type=['max_pool', 'max_pool', 'max_pool', 'max_pool', 'max_pool' ]
    batch_norm=True,
    dropout=False,
    softmax=False,
    gpu_num=4,
    name='sfcn_quantile'
    )

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3')


In [12]:
model.compile(learning_rate=3e-4)

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 182, 218, 182, 1) 0         
_________________________________________________________________
conv_0 (Conv3D)              (None, 91, 109, 91, 32)   896       
_________________________________________________________________
batchnorm_0 (BatchNormalizat (None, 91, 109, 91, 32)   128       
_________________________________________________________________
maxpool_0 (MaxPooling3D)     (None, 45, 54, 45, 32)    0         
_________________________________________________________________
activation_0 (Activation)    (None, 45, 54, 45, 32)    0         
_________________________________________________________________
conv_1 (Conv3D)              (None, 23, 27, 23, 64)    55360     
_________________________________________________________________
batchnorm_1 (BatchNormalizat (None, 23, 27, 23, 64)    256 

In [8]:
model.train_generator(train_gen, valid_gen, batch_size=batch_size, epochs=40)

2021-12-09 18:26:59.922864: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:656] In AUTO-mode, and switching to DATA-based sharding, instead of FILE-based sharding as we cannot find appropriate reader dataset op(s) to shard. Error: Did not find a shardable source, walked to a node which is not a dataset: name: "FlatMapDataset/_2"
op: "FlatMapDataset"
input: "TensorDataset/_1"
attr {
  key: "Targuments"
  value {
    list {
    }
  }
}
attr {
  key: "f"
  value {
    func {
      name: "__inference_Dataset_flat_map_flat_map_fn_1468"
    }
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: -1
        }
        dim {
          size: -1
        }
        dim {
          size: -1
        }
        dim {
          size: -1
        }
        dim {
          size: -1
        }
      }
      shape {
        dim {
          size: -1
        }
        dim {
          size: -1
        }
      }
    }
  }
}
attr {
  key: "output_types"
  va

Epoch 1/40
INFO:tensorflow:batch_all_reduce: 26 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:batch_all_reduce: 26 all-reduces with algorithm = nccl, num_packs = 1


2021-12-09 18:27:12.895487: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.7
2021-12-09 18:27:14.282136: W tensorflow/stream_executor/gpu/asm_compiler.cc:63] Running ptxas --version returned 256
2021-12-09 18:27:14.421731: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] Internal: ptxas exited with non-zero error code 256, output: 
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.
2021-12-09 18:27:15.684059: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10


1569/1569 [==============================] - ETA: 0s - loss: 0.1085 - mae: 0.2716

2021-12-09 18:41:44.207392: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:656] In AUTO-mode, and switching to DATA-based sharding, instead of FILE-based sharding as we cannot find appropriate reader dataset op(s) to shard. Error: Did not find a shardable source, walked to a node which is not a dataset: name: "FlatMapDataset/_2"
op: "FlatMapDataset"
input: "TensorDataset/_1"
attr {
  key: "Targuments"
  value {
    list {
    }
  }
}
attr {
  key: "f"
  value {
    func {
      name: "__inference_Dataset_flat_map_flat_map_fn_17151"
    }
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: -1
        }
        dim {
          size: -1
        }
        dim {
          size: -1
        }
        dim {
          size: -1
        }
        dim {
          size: -1
        }
      }
      shape {
        dim {
          size: -1
        }
        dim {
          size: -1
        }
      }
    }
  }
}
attr {
  key: "output_types"
  v

1569/1569 [==============================] - 1153s 692ms/step - loss: 0.1085 - mae: 0.2716 - val_loss: 0.0964 - val_mae: 0.2621
Epoch 2/40
1569/1569 [==============================] - 1050s 668ms/step - loss: 0.0808 - mae: 0.2454 - val_loss: 0.1126 - val_mae: 0.2773
Epoch 3/40
1569/1569 [==============================] - 1076s 685ms/step - loss: 0.0724 - mae: 0.2306 - val_loss: 0.1819 - val_mae: 0.3410
Epoch 4/40
1569/1569 [==============================] - 1038s 660ms/step - loss: 0.0613 - mae: 0.2085 - val_loss: 0.1657 - val_mae: 0.3242


In [ ]:
#model.load_weights('weights/checkpoint_sfcn_batchnorm')
# train_df.iloc[9]['path']

In [ ]:
#model.model.evaluate(valid_gen)
# train_df['path'].iloc[0]

In [ ]:

#data_size=200
#X = np.random.normal(loc=5, scale=5, size=(data_size, *(input_dim), 1))
#y = np.random.normal(loc=1, scale=1, size=(data_size, num_output))
#model.train(X, y, batch_size, 300)

In [ ]:
#x_test = np.random.normal(loc=8, scale=3, size=(1, 160, 192, 160, 1))

In [ ]:
#model.predict(x_test)

In [ ]:
#y_test = np.mean(x_test, axis=(1,2,3,4))
#print(y_test)